[View in Colaboratory](https://colab.research.google.com/github/sqlwu/image/blob/master/Convolutional_Neural_Network.ipynb)

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
# 定義梯度下降批量
batch_size = 128
# 定義分類數量
num_classes = 10
# 定義訓練週期
epochs = 12

# 定義圖像寬、高
img_rows, img_cols = 28, 28

# 載入 MNIST 訓練資料
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 保留原始資料，供 cross tab function 使用
y_test_org = y_test

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
# channels_first: 色彩通道(R/G/B)資料(深度)放在第2維度，第3、4維度放置寬與高
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else: # channels_last: 色彩通道(R/G/B)資料(深度)放在第4維度，第2、3維度放置寬與高
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [0]:
# 轉換色彩 0~255 資料為 0~1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# y 值轉成 one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# 建立簡單的線性執行的模型
model = Sequential()
# 建立卷積層，filter=32,即 output space 的深度, Kernal Size: 3x3, activation function 採用 relu
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
# 建立卷積層，filter=64,即 output size, Kernal Size: 3x3, activation function 採用 relu
model.add(Conv2D(64, (3, 3), activation='relu'))
# 建立池化層，池化大小=2x2，取最大值
model.add(MaxPooling2D(pool_size=(2, 2)))
# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))
# Flatten層把多維的輸入一維化，常用在從卷積層到全連接層的過渡。
model.add(Flatten())
# 全連接層: 128個output
model.add(Dense(128, activation='relu'))
# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.5
model.add(Dropout(0.5))
# 使用 softmax activation function，將結果分類
model.add(Dense(num_classes, activation='softmax'))

In [6]:
# 編譯: 選擇損失函數、優化方法及成效衡量方式
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# 進行訓練, 訓練過程會存在 train_history 變數中
train_history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 12s 203us/step - loss: 0.2741 - acc: 0.9170 - val_loss: 0.0583 - val_acc: 0.9818
Epoch 2/12
60000/60000 [==============================] - 10s 170us/step - loss: 0.0894 - acc: 0.9733 - val_loss: 0.0491 - val_acc: 0.9846
Epoch 3/12
60000/60000 [==============================] - 10s 168us/step - loss: 0.0665 - acc: 0.9803 - val_loss: 0.0367 - val_acc: 0.9873
Epoch 4/12
60000/60000 [==============================] - 10s 168us/step - loss: 0.0561 - acc: 0.9831 - val_loss: 0.0331 - val_acc: 0.9893
Epoch 5/12
60000/60000 [==============================] - 10s 168us/step - loss: 0.0478 - acc: 0.9858 - val_loss: 0.0301 - val_acc: 0.9898
Epoch 6/12
60000/60000 [==============================] - 10s 170us/step - loss: 0.0419 - acc: 0.9870 - val_loss: 0.0291 - val_acc: 0.9903
Epoch 7/12
60000/60000 [==============================] - 10s 168us/step - loss: 0.0370 - acc: 0.9889 - val_loss: 0.

In [7]:
# 顯示損失函數、訓練成果(分數)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03087429754524378
Test accuracy: 0.9913


In [0]:
# 模型結構存檔
from keras.models import model_from_json
json_string = model.to_json()
with open("cnn.config", "w") as text_file:
    text_file.write(json_string)

    
# 模型訓練結果存檔
model.save_weights("cnn.weight")

In [10]:
!ls

cnn.config  cnn.weight	sample_data


In [11]:
# 計算『混淆矩陣』(Confusion Matrix)，顯示測試集分類的正確及錯認總和數
import pandas as pd 
predictions = model.predict_classes(x_test) 
pd.crosstab(y_test_org, predictions, rownames=['實際值'], colnames=['預測值'])

預測值,0,1,2,3,4,5,6,7,8,9
實際值,,,,,,,,,,
0,978,0,1,0,0,0,0,0,0,1
1,0,1132,1,2,0,0,0,0,0,0
2,2,2,1022,0,1,0,0,5,0,0
3,0,0,2,1004,0,2,0,1,1,0
4,0,0,0,0,975,0,2,0,1,4
5,2,0,1,5,0,883,1,0,0,0
6,5,2,0,1,1,1,946,0,2,0
7,0,2,6,1,0,0,0,1017,1,1
8,4,0,2,0,0,0,1,1,965,1
